In [ ]:
from pathlib import Path
import numpy as np
import numpy.typing as npt
import h5py
import matplotlib.pyplot as plt
from topostats.io import LoadScans
from topostats.plotting import Colormap
import matplotlib.colors as mcolors

colormap = Colormap()
cmap = colormap.get_cmap()
vmin = -3
vmax = 4
square_markersize_figsize_20 = 3.8

from fig_colours_and_functions import (
    good_pixels_colormap,
    good_pixels_alpha,
    good_pixels_colour,
    good_pixels_colour_alpha,
    regular_pixels_colormap,
    regular_pixels_alpha,
    branch_underlying_colour,
    branch_overlying_colour,
    good_mask_color,
    good_mask_alpha,
    molecule_1_colour,
    molecule_2_colour,
    overlay_mask,
    dilate_branches_in_mask,
    plot_overlay_dilated_branches_in_mask,
    overlay_mask_with_specific_colour,
    plot_overlay_dilated_branches_in_mask_specific_colours,
)

In [ ]:
figure_dir = Path("/Users/sylvi/topo_data/topostats_2/figures/fig-disordered-tracing")

In [ ]:
height_pruning_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/height-pruning")
height_pruning_saved_outputs_dir = height_pruning_dir / "saved_outputs"

In [ ]:
# height pruning example
height_pruning_height_pruned_filename = Path("properly-pruned-20231005_unknot_SC_3ng_mgni.0_00023.topostats")
pruning_height_pruned_file = height_pruning_saved_outputs_dir / height_pruning_height_pruned_filename

loadscans = LoadScans([pruning_height_pruned_file], channel="dummy")
loadscans.get_data()
loadscans_image_dicts = loadscans.img_dict

height_pruning_height_pruned_image_data = loadscans_image_dicts[height_pruning_height_pruned_filename.stem]
height_pruning_height_pruned_p2nm = height_pruning_height_pruned_image_data["pixel_to_nm_scaling"]
height_pruning_height_original_skeleton = height_pruning_height_pruned_image_data["disordered_traces"]["above"][
    "grain_0"
]["skeleton"]
height_pruning_height_pruned_skeleton = height_pruning_height_pruned_image_data["disordered_traces"]["above"][
    "grain_0"
]["pruned_skeleton"]
height_pruning_height_pruned_bbox = height_pruning_height_pruned_image_data["disordered_traces"]["above"]["grain_0"][
    "bbox"
]
height_pruning_height_pruned_crop = height_pruning_height_pruned_image_data["image"][
    height_pruning_height_pruned_bbox[0] : height_pruning_height_pruned_bbox[2],
    height_pruning_height_pruned_bbox[1] : height_pruning_height_pruned_bbox[3],
]
height_pruning_height_pruned_mask_crop = height_pruning_height_pruned_image_data["grain_tensors"]["above"][:, :, 1][
    height_pruning_height_pruned_bbox[0] : height_pruning_height_pruned_bbox[2],
    height_pruning_height_pruned_bbox[1] : height_pruning_height_pruned_bbox[3],
]
print(
    f"crop size: {height_pruning_height_pruned_crop.shape} px, {height_pruning_height_pruned_crop.shape[0]*height_pruning_height_pruned_p2nm:.1f} x {height_pruning_height_pruned_crop.shape[1]*height_pruning_height_pruned_p2nm:.1f} nm"
)
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(height_pruning_height_pruned_crop, cmap=cmap, vmin=vmin, vmax=vmax)
mask = np.ma.masked_where(height_pruning_height_pruned_mask_crop == 0, height_pruning_height_pruned_mask_crop)
overlay_mask(mask, good_mask_color)
plt.savefig(figure_dir / "fig-disordered-tracing-height-pruning-mask.png")
plt.show()

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(height_pruning_height_pruned_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(height_pruning_height_original_skeleton == 0, height_pruning_height_original_skeleton),
    cmap=regular_pixels_colormap,
    alpha=1.0,
)
plt.savefig(figure_dir / "fig-disordered-tracing-height-pruning-original.png")
plt.show()

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(height_pruning_height_pruned_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(np.bool(height_pruning_height_pruned_skeleton) == 0, height_pruning_height_pruned_skeleton),
    cmap=good_pixels_colormap,
    alpha=good_pixels_alpha,
)
plt.savefig(figure_dir / "fig-disordered-tracing-height-pruning-pruned.png")
plt.show()

In [ ]:
whole_pruning_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/fig-skeletonisation-pruning-full")
whole_pruning_saved_outputs_dir = whole_pruning_dir / "output_fig_skeletonisation_pruning_example" / "processed"
whole_pruning_filename = Path("20250528_picoz_0RA_tip_0.0_00013.topostats")
whole_pruning_file = whole_pruning_saved_outputs_dir / whole_pruning_filename
loadscans = LoadScans([whole_pruning_file], channel="dummy")
loadscans.get_data()
loadscans_image_dicts = loadscans.img_dict
whole_pruning_image_data = loadscans_image_dicts[whole_pruning_filename.stem]
whole_pruning_p2nm = whole_pruning_image_data["pixel_to_nm_scaling"]
whole_pruning_original_skeleton = whole_pruning_image_data["disordered_traces"]["above"]["grain_0"]["skeleton"]
whole_pruning_pruned_skeleton = whole_pruning_image_data["disordered_traces"]["above"]["grain_0"]["pruned_skeleton"]
whole_pruning_grain_bbox = whole_pruning_image_data["disordered_traces"]["above"]["grain_0"]["bbox"]
whole_pruning_crop = whole_pruning_image_data["image"][
    whole_pruning_grain_bbox[0] : whole_pruning_grain_bbox[2],
    whole_pruning_grain_bbox[1] : whole_pruning_grain_bbox[3],
]
print(
    f"crop size: {whole_pruning_crop.shape} px, {whole_pruning_crop.shape[0]*whole_pruning_p2nm:.1f} x {whole_pruning_crop.shape[1]*whole_pruning_p2nm:.1f} nm"
)

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(whole_pruning_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(whole_pruning_original_skeleton == 0, whole_pruning_original_skeleton),
    cmap=regular_pixels_colormap,
    alpha=1.0,
)
plt.savefig(figure_dir / "fig-disordered-tracing-pruning-full.png")
plt.show()
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(whole_pruning_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(np.bool(whole_pruning_pruned_skeleton) == 0, whole_pruning_pruned_skeleton),
    cmap=good_pixels_colormap,
    alpha=good_pixels_alpha,
)
plt.savefig(figure_dir / "fig-disordered-tracing-pruning-full-pruned.png")
plt.show()

In [ ]:
# height biasing

height_biasing_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/20250528_RA_dose_0GY_picoz")
height_biasing_10_output_file = (
    height_biasing_dir
    / "output_fig_disordered_tracing_height_bias_10"
    / "processed"
    / "20250528_picoz_0RA_tip_0.0_00019.topostats"
)
height_biasing_01_output_file = (
    height_biasing_dir
    / "output_fig_disordered_tracing_height_bias_01"
    / "processed"
    / "20250528_picoz_0RA_tip_0.0_00019.topostats"
)
loadscans_10 = LoadScans([height_biasing_10_output_file], channel="dummy")
loadscans_10.get_data()
loadscans_10_image_dicts = loadscans_10.img_dict
height_biasing_10_image_data = loadscans_10_image_dicts[height_biasing_10_output_file.stem]
height_biasing_10_p2nm = height_biasing_10_image_data["pixel_to_nm_scaling"]
height_biasing_10_mask = height_biasing_10_image_data["grain_tensors"]["above"][:, :, 1]
height_biasing_10_original_skeleton = height_biasing_10_image_data["disordered_traces"]["above"]["grain_1"]["skeleton"]
height_biasing_10_bbox = height_biasing_10_image_data["disordered_traces"]["above"]["grain_1"]["bbox"]
height_biasing_10_crop = height_biasing_10_image_data["image"][
    height_biasing_10_bbox[0] : height_biasing_10_bbox[2],
    height_biasing_10_bbox[1] : height_biasing_10_bbox[3],
]
height_biasing_10_crop_mask = height_biasing_10_mask[
    height_biasing_10_bbox[0] : height_biasing_10_bbox[2],
    height_biasing_10_bbox[1] : height_biasing_10_bbox[3],
]
print(
    f"crop size: {height_biasing_10_crop.shape} px, {height_biasing_10_crop.shape[0]*height_biasing_10_p2nm:.1f} x {height_biasing_10_crop.shape[1]*height_biasing_10_p2nm:.1f} nm"
)
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(height_biasing_10_crop, cmap=cmap, vmin=vmin, vmax=vmax)
overlay_mask(np.ma.masked_where(height_biasing_10_crop_mask == 0, height_biasing_10_crop_mask), good_mask_color)
plt.savefig(figure_dir / "fig-disordered-tracing-height-biasing-10-mask.png")
plt.show()

fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(height_biasing_10_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(height_biasing_10_original_skeleton == 0, height_biasing_10_original_skeleton),
    cmap=regular_pixels_colormap,
    alpha=1.0,
)
plt.savefig(figure_dir / "fig-disordered-tracing-height-biasing-10.png")
plt.show()
loadscans_01 = LoadScans([height_biasing_01_output_file], channel="dummy")
loadscans_01.get_data()
loadscans_01_image_dicts = loadscans_01.img_dict
height_biasing_01_image_data = loadscans_01_image_dicts[height_biasing_01_output_file.stem]
height_biasing_01_p2nm = height_biasing_01_image_data["pixel_to_nm_scaling"]
height_biasing_01_mask = height_biasing_01_image_data["grain_tensors"]["above"][:, :, 1]
height_biasing_01_original_skeleton = height_biasing_01_image_data["disordered_traces"]["above"]["grain_1"]["skeleton"]
height_biasing_01_bbox = height_biasing_01_image_data["disordered_traces"]["above"]["grain_1"]["bbox"]
height_biasing_01_crop = height_biasing_01_image_data["image"][
    height_biasing_01_bbox[0] : height_biasing_01_bbox[2],
    height_biasing_01_bbox[1] : height_biasing_01_bbox[3],
]
print(
    f"crop size: {height_biasing_01_crop.shape} px, {height_biasing_01_crop.shape[0]*height_biasing_01_p2nm:.1f} x {height_biasing_01_crop.shape[1]*height_biasing_01_p2nm:.1f} nm"
)
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(height_biasing_01_crop, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(
    np.ma.masked_where(height_biasing_10_original_skeleton == 0, height_biasing_10_original_skeleton),
    cmap=regular_pixels_colormap,
    alpha=0.4,
)
ax.imshow(
    np.ma.masked_where(height_biasing_01_original_skeleton == 0, height_biasing_01_original_skeleton),
    cmap=good_pixels_colormap,
    alpha=good_pixels_alpha,
)
plt.savefig(figure_dir / "fig-disordered-tracing-height-biasing-01.png")
plt.show()